# Import all SET metadata into MNE object

### ToC
- Import the full Matlab EEG.event struct containning `epoch_id`
- Import SET with MNE (epoched data, basic event structure)
    - Use SciPy's loadmat() to read the struct and make a pd.df ([source](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853)).
- Look into the true event df to find epochs falling into set times

### Imports

In [6]:
import mne
import os
from scipy.io import loadmat

import numpy as np
import pandas as pd

from hypyp import analyses

Define path to data directory and define subject(s)

In [2]:
path = "../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/"

subject_1 = "SAMPLE-1_NS_epochs.set"
subject_2 = "SAMPLE-2_NS_epochs.set"

### Making a custom event df

#### Instantiating the class

Because MNE's function `read_epochs_eeglab()` does all the work of formating the SET file's events df into a MNE-formated df, we lose important information (here, the column `epoch_id`). The class we define bellow allow to keep all these metadata and provide tools to 'cut' it to specific periodes (i.e., a time window) dynamicaly (i.e., a sliding window!). 

Bellow we import a `class` that:
- takes a SET file and read the (full) set of metadata associated to it (see `Class.df`)
- create a `Class.df_sw` which only contains events from `Class.df` which falls between `sw_min` and `sw_max`.
    - if `sw_min` and `sw_max` are defined (floats)
    - or leave `Class.df_sw` as an empty pd.Dataframe if `sw_min` and `sw_max` are `None`

> The function `read_set_events()` that enables reading MatLab's EEG.event struct embeded into a SET file into a panda df is sourced from [here](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) and found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853).

In [3]:
from utils import eeg_sw

Instantiate the class twice for 2 subject.

NB: We're not definin any sliding window parameter at the moment. 

In [4]:
# Channels to keep for analysis 
ch_to_keep_dic = [
    'Fp1', 'Fp2', 'F3', 'F4', 
    'C3', 'C4', 
    'P3', 'P4', 
    'O1', 'O2', 
    'F7', 'F8', 
    'T7', 'T8', 
    'P7', 'P8', 
    'Fz', 'Cz', 'Pz', 
    'AFz', 'CPz', 'POz']

#  We're exluding the channel 'M1' & 'M2'

In [5]:
sub1 = eeg_sw.import_set_custom(path+subject_1, sw_min=None, sw_max=None, ch_to_keep=ch_to_keep_dic)
sub2 = eeg_sw.import_set_custom(path+subject_2, sw_min=None, sw_max=None, ch_to_keep=ch_to_keep_dic)

Extracting parameters from /Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/SAMPLE-1_NS_epochs.set...
Not setting metadata
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from /Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/SAMPLE-2_NS_epochs.set...
Not setting metadata


/Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/utils/eeg_sw.py:48: RuntimeWarning: Data file name in EEG.data (hyper_cleaned_SNS_013L_014S_N_NS_sub_1 epochs.fdt) is incorrect, the file name must have changed on disk, using the correct file name (SAMPLE-1_NS_epochs.fdt).
  self.eeg = mne.io.read_epochs_eeglab(self.path).pick_channels(self.ch_to_keep, ordered=False)
/Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/utils/eeg_sw.py:48: RuntimeWarning: Data file name in EEG.data (hyper_cleaned_SNS_013L_014S_N_NS_sub_2 epochs.fdt) is incorrect, the file name must have changed on disk, using the correct file name (SAMPLE-2_NS_epochs.fdt).
  self.eeg = mne.io.read_epochs_eeglab(self.path).pick_channels(self.ch_to_keep, ordered=False)


Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.


# HERE

Compute the Welch Power Spectral Density

In [8]:
psd1 = analyses.pow(
    sub1.eeg, 
    fmin=7.5, fmax=11, 
    n_fft=1000, 
    n_per_seg=1000, 
    epochs_average=True)
    
psd2 = analyses.pow(
    sub2.eeg, 
    fmin=7.5, fmax=11, 
    n_fft=1000, 
    n_per_seg=1000, 
    epochs_average=True)
    
data_psd = np.array([psd1.psd, psd2.psd])

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


In [21]:
psd1.psd.shape

(22, 8)

# THERE

In [ ]:
sub1.eeg.events.shape

In [ ]:
sub1.df.shape

# below

The event dataframe can now be access using `sub1.df`. However, since we didn't any specific sliding window (sw) parameter, the "cutted" `sub1.df_sw` is left empty.

In [ ]:
sub1.df

#### Select specific events

To select a specific  range of event (e.g., from t=2 to t=16) we're using the `event_slider` method to update the class variable `sub1.df_sw`.

In [ ]:
sub1.event_slider(sw_min=6, sw_max=10)
sub1.df_sw

If we're reusing the `event_slider` method, the `sub1.df_sw` will be updated.

NB: the method supports floats (although there is no need for them here)

In [ ]:
sub1.event_slider(sw_min=24.4, sw_max=36.9)
sub1.df_sw

#### Convert `df_sw` to MNE convention (`df_sw_mne`)

Here we use the `convert_to_MNE_event` to grab `df_sw`, select the column of choice, create the (necessary) `dontuse` column, and convert it to numpy (MNE doesn't take pandas df).

In [ ]:
sub1.

In [ ]:
sub1.convert_EEG_to_MNE()

> NB: the right column is the epoch column, you'll notice that these aren't the same values as the ones we selected with the `event_slider` method. That's because some have been rejected and that causes a shift.

In [ ]:
sub1.df_sw_mne

## Todo

- use `df_sw_mne` and the MNE's metadata method to truely select the epoch of choice and get them ready for analysis

## Archives

In [ ]:
epochs = mne.io.read_epochs_eeglab(path+sub1)

In [ ]:
def read_set_events(filename, ignore_fields=None):
	'''Open set file, read events and turn them into a dataframe
	Parameters
	----------
	filename: str
		Name of the set file to read (absolute or relative path)
	ignore_fields: list of str | None
		Event fields to ignore
	Returns
	-------
	df: pandas.DatFrame
		Events read into a dataframe
	'''
	EEG = loadmat(filename, uint16_codec='latin1',
				  struct_as_record=False, squeeze_me=True)['EEG']
	flds = [f for f in dir(EEG.event[0]) if not f.startswith('_')]
	events = EEG.event
	df_dict = dict()
	for f in flds:
		df_dict[f] = [ev.__getattribute__(f) for ev in events]
	df = pd.DataFrame(df_dict)

	# reorder columns:
	take_fields = ['epoch', 'type']
	ignore_fields = list() if ignore_fields is None else ignore_fields
	take_fields.extend([col for col in df.columns if not
					 (col in take_fields or col in ignore_fields)])
	return df.loc[:, take_fields]

In [ ]:
df = read_set_events(path+sub1)
df

In [ ]:
    

# epochs1 = mne.io.read_epochs_eeglab(path+sub1)
# epochs2 = mne.io.read_epochs_eeglab(path+sub2)

x=0
shapes=list()

# iterate over files in path
for filename in os.listdir(path):
    if filename[-3:] == "set" :
        f = os.path.join(path, filename)
        epochs = mne.io.read_epochs_eeglab(f)
        shapes.append(np.shape(epochs._data))
    else:
        pass



In [ ]:
print(np.mean(shapes, axis=0))
print("Data left after pre-processing (%): ")
print(((np.mean(shapes, axis=0)[0]*100)/(5*60)))

In [ ]:
shapes[2]

In [ ]:
print(np.shape(epochs1.selection))
print(epochs1.selection)

In [ ]:
epochs1.plot(block=True, n_channels=24)